In [1]:
!apt-get update -y
!apt-get install -y \
libglib2.0-0 \
libnss3 \
libfontconfig1 \
gconf2-common \
libgbm1 \
udev \
libu2f-udev
!apt --fix-broken install -y
!wget -P /tmp https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/119.0.6045.105/linux64/chrome-linux64.zip
!unzip /tmp/chrome-linux64.zip -d /usr/bin/
!wget -P /tmp https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/119.0.6045.105/linux64/chromedriver-linux64.zip
!unzip /tmp/chromedriver-linux64.zip -d /usr/bin/
!apt install -y python3-selenium
!pip install selenium==4.7.0
!pip install webdriver_manager

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]                
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease                                              
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]                             
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]             
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [75.2 kB]         
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,607 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]    
Get:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InR

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from lxml import etree

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

In [3]:
# /usr/bin/chrome-linux64/chrome
# /usr/bin/chromedriver-linux64/chromedriver
CHROME_BINARY_LOCATION = "/usr/bin/chrome-linux64/chrome"
CHROMEDRIVER_BINARY_LOCATION = "/usr/bin/chromedriver-linux64/chromedriver"

def add_driver_options(options):
    """
    Add configurable options
    """
    chrome_options = Options()
    for opt in options:
        chrome_options.add_argument(opt)
    return chrome_options

def initialize_driver():
    """
    Initialize the web driver
    """
    driver_config = {
        "options": [
            "--headless",
            "--no-sandbox",
            "--allow-insecure-localhost",
            "--disable-dev-shm-usage",
            # "--incognito",
            "--window-size=1920x1080",
            "user-agent=Chrome/119.0.6045.105"
        ],
    }
    options = add_driver_options(driver_config["options"])
    options.binary_location = CHROME_BINARY_LOCATION
    service = Service(executable_path=CHROMEDRIVER_BINARY_LOCATION)
    driver = webdriver.Chrome(
        service=service,
        options=options)
    return driver


In [6]:
import json
import os
import re
import time
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
from bs4 import BeautifulSoup
from lxml import etree
import requests
from dateutil.relativedelta import relativedelta
from datetime import datetime, timedelta

class Crawler:
    def __init__(self):
        # lấy root
        self.home_page = f"https://thuvienphapluat.vn/hoi-dap-phap-luat/tim-tu-van"
        self.root = f"https://thuvienphapluat.vn/hoi-dap-phap-luat"
        self.head_page = [
            f"https://thuvienphapluat.vn/hoi-dap-phap-luat/tim-tu-van?page={i}"
            for i in range(1, 20)
        ]

    # lấy tất cả link trong 1 page
    def get_pages(self, url):
        self.driver.get(url)
        time.sleep(3)  # đợi trang load nếu cần
        bs = BeautifulSoup(self.driver.page_source, 'html.parser')
    
        link_page = []
        seen = set()
    
        try:
            # lấy tất cả link ở trong page đấy, chưa vào link
            articles = bs.find_all("article", class_="news-card tvpl-find")
            for article in articles:
                a_tag = article.find("a", href=True)
                if a_tag:
                    href = a_tag["href"]
                    #print(href)
                    
                    if href not in seen:
                        seen.add(href)
                        full_link = href if href.startswith("http") else self.root + href
                        link_page.append(full_link)
        except Exception as e:
            print(f"❌ Error extracting links: {e}")
    
        print(f"✅ Found {len(link_page)} pages")
        return link_page

    # lấy content trong mỗi link
    def get_law_content(self, page_source, page_link):
        data = {}
        bs = BeautifulSoup(page_source, 'html.parser')
    
        try:
            section = bs.find("section", id="news-content") # tìm đúng section
            if not section:
                raise ValueError("Không tìm thấy section id='news-content'")
    
            # Lấy question 
            strong = section.find("strong")
            data["question"] = strong.get_text(strip=True) if strong else "N/A"
    
            # Lấy context từ các p
            paragraphs = section.find_all("p")
            context = [p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True)]
    
            data["content"] = "\n".join(context) if context else "N/A"
    
        except Exception as e:
            print(f"Error retrieving law content: {e}")
            data["question"] = "N/A"
            data["content"] = "N/A"
    
        print(f"Crawled from: {page_link}")
        return data


    def prepare_driver(self):
        self.driver = initialize_driver()

    # chạy
    # def run(self, num_pages):
    #     main_url = self.head_page[num_pages]
    
    #     if os.path.exists("data/index.json"):
    #         with open("data/index.json") as f:
    #             index = set(json.load(f))
    #     else:
    #         index = set()
    
    #     try:
    #         self.driver.get(main_url)
    #         time.sleep(3)
        #     soup = BeautifulSoup(self.driver.page_source, 'html.parser')
    
        #     print(self.driver.current_url)
        #     list_link_pages = self.get_pages(main_url)
        #     print(len(list_link_pages), 'pages')
    
        #     # File jsonl đầu ra
        #     with open("data/law_data.jsonl", "w", encoding="utf-8") as fout:
        #         for page_link in list_link_pages:
        #             if page_link in index:
        #                 continue  # Skip if already crawled
    
        #             print(f"Crawling {page_link}")
        #             self.driver.get(page_link)
        #             time.sleep(1)
    
        #             data = self.get_law_content(self.driver.page_source, page_link)
    
        #             if data.get('question') and data.get('content'):
        #                 fout.write(json.dumps({
        #                     "question": data['question'],
        #                     "context": data['content']
        #                 }, ensure_ascii=False) + "\n")
        #                 index.add(page_link)
        #     output_file = f"data/law_data_page_{i}.json"
        #     with open(output_file, "w", encoding="utf-8") as f:
        #         json.dump(page_data, f, ensure_ascii=False, indent=2)
        #     # Ghi lại index đã crawl
        #     with open("data/index.json", "w", encoding="utf-8") as f:
        #         json.dump(list(index), f, ensure_ascii=False)
    
        #     self.driver.delete_all_cookies()
    
        # except Exception as e:
        #     print(f"Error: {e}")
    def run(self, i):
        main_url = f"https://thuvienphapluat.vn/hoi-dap-phap-luat/tim-tu-van?page={i}"
    
        if os.path.exists("data/index.json"):
            with open("data/index.json") as f:
                index = set(json.load(f))
        else:
            index = set()
    
        all_data_file = "data/law_data_all.json"
        if os.path.exists(all_data_file):
            with open(all_data_file, "r", encoding="utf-8") as f:
                all_data = json.load(f)
        else:
            all_data = []
    
        try:
            self.driver.get(main_url)
            time.sleep(3)
            print(self.driver.current_url)
    
            list_link_pages = self.get_pages(main_url)
            print(f"🔗 Found {len(list_link_pages)} links on page {i}")
    
            # Đây là dữ liệu chỉ của page i
            page_data = []
    
            for page_link in list_link_pages:
                if page_link in index:
                    continue
    
                print(f"📝 Crawling {page_link}")
                self.driver.get(page_link)
                time.sleep(1)
    
                data = self.get_law_content(self.driver.page_source, page_link)
    
                if data.get('question') and data.get('content'):
                    record = {
                        "question": data['question'],
                        "context": data['content']
                    }
                    page_data.append(record)
                    all_data.append(record)
                    index.add(page_link)
    
            # Ghi riêng dữ liệu của trang i (nếu bạn vẫn muốn)
            # output_file = f"data/law_data_page_{i}.json"
            # with open(output_file, "w", encoding="utf-8") as f:
            #     json.dump(page_data, f, ensure_ascii=False, indent=2)
    
            # Ghi tổng tất cả
            with open(all_data_file, "w", encoding="utf-8") as f:
                json.dump(all_data, f, ensure_ascii=False, indent=2)
    
            # Ghi index
            with open("data/index.jsonl", "w", encoding="utf-8") as f:
                json.dump(list(index), f, ensure_ascii=False)
    
            self.driver.delete_all_cookies()
    
        except Exception as e:
            print(f"❌ Error at page {i}: {e}")


In [ ]:
import time
import os

if __name__ == "__main__":
    folder_name = 'data'
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    crawler = Crawler()
    crawler.prepare_driver()
    for i in range(1, 500):
        print(f"\n🚀 Crawling Page {i}")
        data = crawler.run(i)  
        print(f"✅ Page {i} done")

    print("All data crawled and saved!")



🚀 Crawling Page 1
https://thuvienphapluat.vn/hoi-dap-phap-luat/tim-tu-van?page=1
✅ Found 20 pages
🔗 Found 20 links on page 1
📝 Crawling https://thuvienphapluat.vn/hoi-dap-phap-luat/19C38-hd-xe-dap-may-chay-trong-ham-duong-bo-khong-co-den-bi-xu-phat-the-nao-ke-tu-ngay-01-08-2016.html
Crawled from: https://thuvienphapluat.vn/hoi-dap-phap-luat/19C38-hd-xe-dap-may-chay-trong-ham-duong-bo-khong-co-den-bi-xu-phat-the-nao-ke-tu-ngay-01-08-2016.html
📝 Crawling https://thuvienphapluat.vn/hoi-dap-phap-luat/19C39-hd-xe-tho-so-chay-trong-ham-duong-bo-khong-co-den-bi-xu-phat-the-nao-ke-tu-ngay-01-08-2016.html
Crawled from: https://thuvienphapluat.vn/hoi-dap-phap-luat/19C39-hd-xe-tho-so-chay-trong-ham-duong-bo-khong-co-den-bi-xu-phat-the-nao-ke-tu-ngay-01-08-2016.html
📝 Crawling https://thuvienphapluat.vn/hoi-dap-phap-luat/19C3A-hd-chau-co-nghĩa-vu-cap-duong-nuoi-dì-neu-khong-co-nguoi-cap-duong.html
Crawled from: https://thuvienphapluat.vn/hoi-dap-phap-luat/19C3A-hd-chau-co-nghĩa-vu-cap-duong-nu